In [1]:
# Data handling libraries
import pandas as pd
from pandas import json_normalize


In [2]:
# Load the training data from a JSON Lines file (one JSON object per line)
train_data = pd.read_json('Kaggle2025/train.jsonl', lines=True)
# The tweet data is nested. json_normalize flattens the nested JSON into columns.
train_data = json_normalize(train_data.to_dict(orient='records'))

# Load the Kaggle test data (which we will make predictions on)
kaggle_data = pd.read_json('Kaggle2025/kaggle_test.jsonl', lines=True)
# Also normalize the Kaggle data
kaggle_data = json_normalize(kaggle_data.to_dict(orient='records'))


# Separate features from the target variable for the training set
X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

X_kaggle = kaggle_data

In [13]:
cols = list(X_kaggle.columns)
n = len(cols)
k = 4  # taille du bloc

def missing_ratio(col):
    return X_kaggle[col].isna().mean()   # proportion (0 → 1)

for i in range(n // k):
    block = cols[i*k : (i+1)*k]
    annotated = [f"{c} {missing_ratio(c):.2%}" for c in block]
    for c in annotated:
        print(c, end=' | ')
    print()

# bloc final (si n n'est pas divisible par k)
last_block = cols[(n // k) * k :]
if last_block:
    annotated = [f"{c} ({missing_ratio(c):.2%} missing)" for c in last_block]
    for c in annotated:
        print(c, end=' | ')
    print()


in_reply_to_status_id_str 69.67% | in_reply_to_status_id 69.67% | created_at 0.00% | in_reply_to_user_id_str 68.30% | 
source 0.00% | retweet_count 0.00% | retweeted 0.00% | geo 100.00% | 
filter_level 0.00% | in_reply_to_screen_name 68.30% | is_quote_status 0.00% | id_str 0.00% | 
in_reply_to_user_id 68.30% | favorite_count 0.00% | text 0.00% | place 100.00% | 
lang 0.00% | quote_count 0.00% | favorited 0.00% | possibly_sensitive 68.23% | 
coordinates 100.00% | truncated 0.00% | timestamp_ms 0.00% | reply_count 0.00% | 
contributors 100.00% | challenge_id 0.00% | extended_entities 100.00% | display_text_range 61.04% | 
quoted_status 100.00% | quoted_status_id 65.77% | quoted_status_permalink 100.00% | quoted_status_id_str 65.77% | 
extended_tweet.entities.urls 49.80% | extended_tweet.entities.hashtags 49.80% | extended_tweet.entities.user_mentions 49.80% | extended_tweet.entities.symbols 49.80% | 
extended_tweet.full_text 49.80% | extended_tweet.display_text_range 49.80% | entities.ur

In [4]:
import lightgbm as lgb
import pandas as pd

# X : DataFrame de shape (N, 194)
# y : labels (0/1)

train_data = lgb.Dataset(X_train, label=y_train)

params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": -1,
}

model = lgb.train(params, train_data, num_boost_round=300)

# importance des features
importances = model.feature_importance()
feat_names = X_train.columns

df_imp = pd.DataFrame({
    "feature": feat_names,
    "importance": importances
}).sort_values("importance", ascending=False)

print(df_imp.head(20))


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: created_at: datetime64[ns, UTC], source: object, filter_level: object, in_reply_to_screen_name: object, text: object, lang: object, timestamp_ms: datetime64[ns], display_text_range: object, withheld_in_countries: object, quoted_status.extended_tweet.entities.urls: object, quoted_status.extended_tweet.entities.hashtags: object, quoted_status.extended_tweet.entities.user_mentions: object, quoted_status.extended_tweet.entities.symbols: object, quoted_status.extended_tweet.full_text: object, quoted_status.extended_tweet.display_text_range: object, quoted_status.in_reply_to_status_id_str: object, quoted_status.created_at: object, quoted_status.in_reply_to_user_id_str: object, quoted_status.source: object, quoted_status.retweeted: object, quoted_status.filter_level: object, quoted_status.in_reply_to_screen_name: object, quoted_status.is_quote_status: object, quoted_status.id_str: object, quoted_status.text: 